In [7]:
option = 'H100_80GB_HBM3'
!mkdir -p notes/capture/{option}
!mkdir -p notes/data/{option}
!meson test -C build/ hamiltonian --verbose -t 0 > notes/capture/{option}/gpu.txt

In [8]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

# Read the file content
def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+structure\s+(?P<structure>.+)\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_transfer_time\s+(?P<gpu_transfer_time>[\d.]+)"

  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)

  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
    'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh',
    'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
    'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_transfer_time'
  ]
  # # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

df = parse_to_datafame(f"notes/capture/{option}/gpu.txt")
df.to_csv(f"notes/data/{option}/gpu.csv", index=False)
df

,structure,nat,nprim,maxl,max_nnl,mean_nnl,max_nsh,cpu_time,gpu_gb_in,gpu_gb_out,gpu_gb_total,gpu_between_atoms,gpu_in_atoms,gpu_time,gpu_transfer_time
0,GLU,29,6,1,28,14,2,25.599215,0.000453,0.000444,0.000896,5.177216,0.348288,5.525504,1.093344
4,alkane_64,192,6,1,30,20,2,38.637041,0.013026,0.012976,0.026002,0.495456,0.298880,0.794336,7.392961
5,alkane_128,384,6,1,30,21,2,38.625414,0.052005,0.051905,0.103909,0.603136,0.274432,0.877568,21.010399
1,dna,387,6,2,165,69,3,44.083036,0.232068,0.231803,0.463871,3.571200,0.805408,4.376608,84.105629
6,alkane_256,768,6,1,30,21,2,54.160045,0.207819,0.207618,0.415437,0.977632,0.271296,1.248928,97.602722
2,1lyz-lysozyme,1102,6,2,309,99,3,239.119671,1.749900,1.748891,3.498791,9.492480,0.845568,10.338048,659.180969
3,103l-hydrolase,1404,6,2,312,99,3,338.086432,2.816439,2.815150,5.631588,11.214592,0.857408,12.072000,1044.699097
7,alkane_512,1536,6,1,30,21,2,113.497335,0.830875,0.830472,1.661347,1.503488,0.508096,2.011584,369.374634
8,alkane_1024,3072,6,1,30,21,2,329.079122,3.322695,3.321889,6.644584,2.708224,0.590464,3.298688,1438.097656
9,alkane_2048,6144,6,1,30,21,2,1725.059579,13.289168,13.287555,26.576724,4.671840,0.517344,5.189184,5091.389160


In [9]:
!nproc
!lscpu

128
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 57 bits virtual
  Byte Order:             Little Endian
CPU(s):                   128
  On-line CPU(s) list:    0-127
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) Gold 6448Y
    CPU family:           6
    Model:                143
    Thread(s) per core:   2
    Core(s) per socket:   32
    Socket(s):            2
    Stepping:             8
    CPU max MHz:          4100.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4200.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc cpuid aperfmperf tsc_known_freq p

In [1]:
!nvidia-smi

Sun May 25 16:19:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:68:00.0 Off |                    0 |
| N/A   45C    P0             72W /  700W |       1MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----